In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/tgs-salt-identification-challenge

In [ ]:
data_folder = '/kaggle/input/tgs-salt-identification-challenge'

In [ ]:
train_df = pd.read_csv('{}/{}'.format(data_folder, 'train.csv'))

In [ ]:
train_df['rle_mask'].fillna(-1, inplace=True)
train_df['binary'] = train_df.apply(lambda row: row.rle_mask != -1, axis=1)

In [ ]:
train_df.head()

In [ ]:
train_df.loc[2,'rle_mask']

In [ ]:
train_df.loc[2,'id']

In [ ]:
!unzip '/kaggle/input/tgs-salt-identification-challenge/train.zip'

In [ ]:
!ls ./

In [ ]:
from IPython.display import Image
Image(filename='./images/a266a2a9df.png')

In [ ]:
Image(filename='./masks/a266a2a9df.png')

In [ ]:
def pad_zeros(array, MAX_WIDTH=128, MAX_HEIGHT=128):
    #assert array.size == (101, 101)
    padded_image = np.zeros(shape=(128,128))
    padded_image[13:114, 13:114] = array
    #assert sum(padded_image) > 0
    return padded_image

In [ ]:
import PIL.Image
import matplotlib.pyplot as plt

In [ ]:
original_image =  np.asarray(PIL.Image.open('./images/a266a2a9df.png').convert('L'),dtype =np.uint8)
print(original_image.shape)
padded_image = pad_zeros(original_image)
plt.subplot("221")
plt.imshow(original_image,cmap = plt.cm.gray)
plt.subplot("222")
plt.imshow(padded_image, cmap =plt.cm.gray)
plt.show()

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.train_frame)
        
    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
            #idx = idx.tolist()
            
        Id = self.train_frame.loc[idx, 'id']
        feature = pad_zeros(np.asarray(Image.open('{}/images/{}.png'.format(self.root_dir, Id)).convert('L'), dtype =np.uint8))
        label   = pad_zeros(np.asarray(Image.open('{}/masks/{}.png'.format(self.root_dir, Id)).convert('1'),dtype =np.uint8))
        sample = {'feature': feature, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

    
class BinaryDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.train_frame['rle_mask'].fillna(-1, inplace=True)
        self.train_frame['binary'] = train_df.apply(lambda row: row.rle_mask != -1, axis=1)
        
    def __len__(self):
        return len(self.train_frame)
        
    def __getitem__(self, idx):
        #if torch.is_tensor(idx):
            #idx = idx.tolist()
            
        Id = self.train_frame.loc[idx, 'id']
        feature = pad_zeros(np.asarray(Image.open('{}/images/{}.png'.format(self.root_dir, Id)).convert('L'), dtype =np.uint8))
        feature = np.expand_dims(feature, axis=0)
        label   = int(self.train_frame.loc[idx,'binary'])
        sample = {'feature': feature, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
def get_item(idx):
    
    Id = train_df.loc[idx, 'id']
    root_dir = "./"
    feature = np.asarray(PIL.Image.open('{}/images/{}.png'.format(root_dir, Id)).convert('L'), dtype =np.uint8)
    label   = np.asarray(PIL.Image.open('{}/masks/{}.png'.format(root_dir, Id)).convert('1'),dtype =np.uint8)
    return {'feature': feature, 'label': label}

In [ ]:
imgDataset = ImageDataset('/kaggle/input/tgs-salt-identification-challenge/train.csv','./')

In [ ]:
def display_sample(id):
    import matplotlib.pyplot as plt
    sample = imgDataset[id]
    plt.subplot("121")
    plt.imshow(sample['feature'],cmap = plt.cm.gray)
    plt.subplot("122")
    plt.imshow(sample['label'], cmap =plt.cm.gray)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
sample = imgDataset[1]
plt.imshow(sample['feature'], cmap=plt.cm.gray)

In [ ]:
display_sample(1)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(imgDataset, batch_size=4,
                        shuffle=True, num_workers=0)

In [ ]:
from torchvision import transforms, utils

def show_batch(sample_batched):
    """Show image with landmarks for a batch of samples."""
    images_batch, labels_batch = \
            sample_batched['feature'], sample_batched['label']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    
    fig, ax = plt.subplots(nrows=batch_size, ncols =2)
    for i, row in enumerate(ax):
        plt.subplot(batch_size, 2, i*2+1)
        plt.imshow(images_batch[i], cmap = plt.cm.gray)
        plt.subplot(batch_size, 2, i*2+2)
        plt.imshow(labels_batch[i], plt.cm.gray)
    #plt.title('Batch from dataloader')

In [ ]:
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['feature'].size(),
          sample_batched['label'].size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break

In [ ]:
bDataset = BinaryDataset('/kaggle/input/tgs-salt-identification-challenge/train.csv','./')
bDataset[5]

In [ ]:
binaryDataLoader = DataLoader(bDataset, batch_size=16,
                        shuffle=True, num_workers=4)

In [ ]:
for i_batch, sample_batched in enumerate(binaryDataLoader):
    #print(sample_batched)
    #print(i_batch, sample_batched['feature'],sample_batched['label'])
    print(sample_batched['feature'].size(), sample_batched['label'])
    break

In [ ]:
import torch
#assert '.'.join(torch.__version__.split('.')[:2]) == '1.6'
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels, stride=1, kernel_size=3, padding=1,bias=False):
        super(ResidualBlock,self).__init__()

        self.cnn1 =nn.Sequential(
            nn.Conv2d(in_channels, middle_channels, 1, stride, padding=0, bias=False),
            nn.BatchNorm2d(middle_channels),
            nn.ReLU(True)
        )

        self.cnn2 = nn.Sequential(
            nn.Conv2d(middle_channels, middle_channels, 3, 1, padding, bias=False),
            nn.BatchNorm2d(middle_channels),
            nn.ReLU(True)
        )

        self.cnn3 = nn.Sequential(
            nn.Conv2d(middle_channels, out_channels, 1, 1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()
            
    def forward(self,x):
        residual = x
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x += self.shortcut(residual)
        x = nn.ReLU(True)(x)
        return x
    
    
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50,self).__init__()
        
#           (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#   (relu): ReLU(inplace=True)
#   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d((3,3), stride=2, dilation=1, padding=1)
        )
        
        self.block2 = nn.Sequential(
            ResidualBlock(64, 64, 256),
            ResidualBlock(256, 64, 256),
            ResidualBlock(256, 64, 256)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(256, 128, 512, stride=2),
            ResidualBlock(512, 128, 512),
            ResidualBlock(512, 128, 512),
            ResidualBlock(512, 128, 512)
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(512, 256, 1024, stride=2),
            ResidualBlock(1024, 256, 1024),
            ResidualBlock(1024, 256, 1024),
            ResidualBlock(1024, 256, 1024)
        )

        self.block5 = nn.Sequential(
            ResidualBlock(1024, 512, 2048, stride=2),
            ResidualBlock(2048, 512, 2048),
            ResidualBlock(2048, 512, 2048),
            ResidualBlock(2048, 512, 2048)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

        self.fc1 = nn.Linear(2048, 1000)

        self.fc2 = nn.Linear(1000, 2)
    
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x1 = self.fc2(F.relu(self.fc1(x)))

        return x1

resnet = ResNet50()

In [ ]:
a = resnet.to(device)

#summary(resnet, (1, 128, 128))

In [ ]:
#check binary or not
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def check_accuracy_part34(loader, model):
#     if loader.dataset.train:
#         print('Checking accuracy on validation set')
#     else:
#         print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for sampled_batch in loader:
            x = sampled_batch['feature']
            y = sampled_batch['label']
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        

def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print("***********", e, "*************")
        for t, sampled_batch in enumerate(binaryDataLoader):
            model.train()  # put model to training mode
            x = sampled_batch['feature']
            y = sampled_batch['label']
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(binaryDataLoader, model)
                print()

In [ ]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
learning_rate = 1e-2
model = None
optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

model = resnet
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

# You should get at least 70% accuracy
train_part34(model, optimizer, epochs=8)